In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import datajoint as dj
import tensorflow as tf
import numpy as np

import seaborn as sns
sns.set_style('ticks', rc={'image.cmap': 'bwr'})

import os
import sys
import inspect
import scipy

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
schema = dj.schema('aecker_mesonet_parameters2', locals())

Connecting aecker@52.202.15.166:3306


In [3]:
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet.parameters import Core, Readout, Model, RegPath, Fit
from cnn_sys_ident.mesonet import MODELS

In [4]:
data_key = dict(data_hash='cfcd208495d565ef66e7dff9f98764da')
model_rel = MODELS['HermiteTransfer'] * MultiDataset() & data_key

# Populate database

In [ ]:
Fit().populate(model_rel, reserve_jobs=True, suppress_errors=True, order='random')

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


  50 | Loss: 4054.25
 100 | Loss: 4043.27
 150 | Loss: 4034.93
 200 | Loss: 4027.61
 250 | Loss: 4025.22
 300 | Loss: 4020.33
 350 | Loss: 4013.03
 400 | Loss: 4008.98
 450 | Loss: 4003.91
 500 | Loss: 4001.34
 550 | Loss: 3997.78
 600 | Loss: 3992.25
 650 | Loss: 3989.99
 700 | Loss: 3988.98
 750 | Loss: 3983.52
 800 | Loss: 3983.09
 850 | Loss: 3982.04
 900 | Loss: 3976.21
 950 | Loss: 3968.98
1000 | Loss: 3969.72
1050 | Loss: 3953.77
1100 | Loss: 3945.67
1150 | Loss: 3938.32
1200 | Loss: 3933.66
1250 | Loss: 3924.95
1300 | Loss: 3937.65
1350 | Loss: 3916.31
1400 | Loss: 3897.95
1450 | Loss: 3882.59
1500 | Loss: 3952.35
1550 | Loss: 3872.50
1600 | Loss: 3847.26
1650 | Loss: 3842.64
1700 | Loss: 3857.84
1750 | Loss: 3843.39
1800 | Loss: 3826.85
1850 | Loss: 3831.16
1900 | Loss: 3810.42
1950 | Loss: 3813.26
2000 | Loss: 3801.65
2050 | Loss: 3892.44
2100 | Loss: 3878.66
2150 | Loss: 3810.98
2200 | Loss: 3816.68
2250 | Loss: 3877.05
INFO:tensorflow:Restoring parameters from /gpfs01/bethg

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


  50 | Loss: 4005.88
 100 | Loss: 3938.65
 150 | Loss: 3922.54
 200 | Loss: 3863.01
 250 | Loss: 3873.45
 300 | Loss: 3904.43
 350 | Loss: 3935.06
 250 | Loss: 3834.09
 300 | Loss: 3836.99
 350 | Loss: 3822.83
 400 | Loss: 3817.44
 450 | Loss: 3821.42
 500 | Loss: 3822.24
 550 | Loss: 3811.39
 600 | Loss: 3819.93
 650 | Loss: 3812.40
 700 | Loss: 3803.51
 750 | Loss: 3804.02
 800 | Loss: 3800.59
 850 | Loss: 3804.45
 900 | Loss: 3820.41
1050 | Loss: 3808.02
INFO:tensorflow:Restoring parameters from /gpfs01/bethge/home/aecker/lab/projects/microns/cnn-sys-ident/checkpoints/aecker_mesonet_data/af91cb9e0c3cbf871172e5cccb6c023f/model.ckpt
Reducing learning rate to 0.000020


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


  50 | Loss: 4006.07
 100 | Loss: 3935.89
 150 | Loss: 3968.48
 200 | Loss: 3858.40
 250 | Loss: 3867.44
 300 | Loss: 3848.94
 350 | Loss: 3854.48
 400 | Loss: 3841.85
 450 | Loss: 3846.01
 500 | Loss: 3840.82
 550 | Loss: 3868.19
 600 | Loss: 3886.90
 650 | Loss: 3880.95
 550 | Loss: 3829.13
 600 | Loss: 3834.11
 650 | Loss: 3822.51
 700 | Loss: 3827.75
 750 | Loss: 3825.34
 800 | Loss: 3828.43
 850 | Loss: 3818.13
 900 | Loss: 3816.67
 950 | Loss: 3818.13
1000 | Loss: 3840.46
1050 | Loss: 3831.44


# Plot results

In [ ]:
data = model.base.data
inputs, responses = data.test()
feed_dict = {model.base.inputs: inputs, model.base.is_training: False}
predictions = model.base.evaluate(model.predictions, feed_dict=feed_dict)
rho = [scipy.stats.pearsonr(p, r)[0] for p, r in zip(predictions.T, responses.T)]

# Manual exploration

In [27]:
from cnn_sys_ident.architectures.cores import StackedRotEquiHermiteConv2dCore
from cnn_sys_ident.architectures.readouts import SpatialXFeatureJointL1Readout, SpatialSparseXFeatureDenseReadout
from cnn_sys_ident.architectures.models import BaseModel, CorePlusReadoutModel
from cnn_sys_ident.architectures.training import Trainer

In [38]:
data = (MultiDataset() & data_key).load_data()

In [37]:
from cnn_sys_ident.architectures.utils import soft_threshold, inv_soft_threshold, sta_init

class SpatialXFeatureJointL1TransferReadout:
    def __init__(self,
                 base,
                 inputs,
                 k_transfer,
                 positive_feature_weights=False,
                 readout_sparsity=0.017,
                 init_masks='sta',
                 scope='readout',
                 reuse=False,
                 **kwargs):
        with base.tf_session.graph.as_default():
            with tf.variable_scope(scope, reuse=reuse):
                data = base.data
                _, num_px_y, num_px_x, num_features = inputs.shape.as_list()
                num_neurons = data.num_neurons

                # masks
                if init_masks == 'sta':
                    images_train, responses_train = data.train()
                    k = (images_train.shape[1] - num_px_y) // 2
                    mask_init = sta_init(images_train, responses_train,
                                         max_val=0.01, sd=0.001)[:,k:-k,k:-k]
                    mask_init = tf.constant_initializer(mask_init)
                else:
                    mask_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
                self.masks = tf.get_variable(
                    'masks',
                    shape=[num_neurons, num_px_y, num_px_x],
                    initializer=mask_init)
                self.masks = tf.abs(self.masks, name='positive_masks')

                # split masks into neurons used for training the core and neurons
                # used for transfer learning (i.e. only readout weights are trained,
                # but they do not affect the core). We then insert a stop_gradient
                # for the units used only for transfer learning, apply the masks
                # and put things back together. This way, there is no gradient flow
                # from the units used for transfer learning back into the core.
                idx_train = tf.range(num_neurons, delta=k_transfer)
                idx_transfer, _ = tf.setdiff1d(tf.range(num_neurons), idx_train)
                idx_all = tf.invert_permutation(tf.concat([idx_train, idx_transfer], axis=0))
                masks_train = tf.gather(self.masks, idx_train)
                masks_transfer = tf.gather(self.masks, idx_transfer)
                masked_train = tf.tensordot(masks_train, (inputs),
                                            [[1, 2], [1, 2]], name='masked_train')
                masked_transfer = tf.tensordot(masks_transfer, tf.stop_gradient(inputs),
                                               [[1, 2], [1, 2]], name='masked_transfer')
                self.masked = tf.transpose(tf.gather(tf.concat(
                            [masked_train, masked_transfer], axis=0), 
                        idx_all, name='masked'),
                    [1, 2, 0])

                # feature weights
                self.feature_weights = tf.get_variable(
                    'feature_weights',
                    shape=[num_neurons, num_features],
                    initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01))
                if positive_feature_weights:
                    self.feature_weights = tf.abs(self.feature_weights, name='positive_feature_weights')
                self.h = tf.reduce_sum(self.masked * tf.transpose(self.feature_weights), 1)

                # L1 regularization for readout layer
                self.readout_reg = readout_sparsity * tf.reduce_sum(
                    tf.reduce_sum(tf.abs(self.masks), [1, 2]) * \
                    tf.reduce_sum(tf.abs(self.feature_weights), 1))
                tf.losses.add_loss(self.readout_reg, tf.GraphKeys.REGULARIZATION_LOSSES)

                # bias and output nonlinearity
                _, responses = data.train()
                bias_init = 0.5 * inv_soft_threshold(responses.mean(axis=0))
                self.biases = tf.get_variable(
                    'biases',
                    shape=[num_neurons],
                    initializer=tf.constant_initializer(bias_init))
                self.output = tf.identity(soft_threshold(self.h + self.biases), name='output')


In [41]:
base = BaseModel(
    data,
    log_dir='test-checkpoints',
    log_hash='testing_transfer'
)
core = StackedRotEquiHermiteConv2dCore(
    base,
    base.inputs,
    num_filters=[16, 16, 16],
    filter_size = [13, 5, 5],
    stride=[1, 1, 1],
    rate=[1, 1, 1],
    padding=['SAME', 'SAME', 'SAME'],
    rel_smooth_weight=[1, 0.5, 0.5],
    rel_sparse_weight=[0, 1, 1],
    activation_fn=['soft', 'soft', 'none'],
    num_rotations=8,
    shared_biases=False,
    conv_smooth_weight=0.025,
    conv_sparse_weight=0.015,
)
readout = SpatialXFeatureJointL1TransferReadout(
    base,
    core.output,
    k_transfer=2,
    positive_feature_weights=False,
    init_masks='rand',
    readout_sparsity=0.017,
)
model = CorePlusReadoutModel(base, core, readout)
trainer = Trainer(base, model)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [42]:
# Using 50% of the neurons for training
iter_num, val_loss, test_corr = trainer.fit(val_steps=50, learning_rate=0.002, lr_decay_steps=2)

  50 | Loss: 4047.85
 100 | Loss: 4022.06
 150 | Loss: 3993.97
 200 | Loss: 3954.39
 250 | Loss: 3925.02
 300 | Loss: 3901.83
 350 | Loss: 3890.11
 400 | Loss: 3900.95
 450 | Loss: 3867.69
 500 | Loss: 3870.01
 550 | Loss: 3877.92
 600 | Loss: 3842.42
 650 | Loss: 3799.27
 700 | Loss: 3772.50
 750 | Loss: 3783.61
 800 | Loss: 3770.68
 850 | Loss: 3884.77
 900 | Loss: 3761.20
 950 | Loss: 3733.74
1000 | Loss: 3752.91
1050 | Loss: 3731.60
1100 | Loss: 3779.30
1150 | Loss: 3829.70
1200 | Loss: 3732.05
1250 | Loss: 3815.78
1300 | Loss: 3770.25
INFO:tensorflow:Restoring parameters from /gpfs01/bethge/home/aecker/lab/projects/microns/cnn-sys-ident/test-checkpoints/testing_transfer/model.ckpt
Reducing learning rate to 0.000200
1100 | Loss: 3713.11
1150 | Loss: 3697.52
1200 | Loss: 3696.02
1250 | Loss: 3685.33
1300 | Loss: 3682.64
1350 | Loss: 3680.14
1400 | Loss: 3678.29
1450 | Loss: 3680.73
1500 | Loss: 3674.82
1550 | Loss: 3674.42
1600 | Loss: 3672.43
1650 | Loss: 3674.66
1700 | Loss: 3671.